In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:

In [3]:
attendance = pd.read_csv('attendance.csv')

attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [4]:
# Rename column something easier to read and call
attendance = attendance.rename(columns={'Unnamed: 0': 'student'})

attendance.head()

,student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [5]:
# Replace letter grades with numbers that can be used in calculations
attendance = attendance.replace(['P', 'T', 'H', 'A'], [1, .9, .5, 0])

attendance

,student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,1,0.9,0.9,0.5,1.0,0.0,0.9,0.9
1,Jane,0,1.0,0.9,0.9,0.9,0.9,0.0,0.9
2,Billy,0,0.9,0.0,0.0,0.5,0.9,1.0,0.9
3,John,1,0.9,0.5,1.0,1.0,0.9,1.0,1.0


In [6]:
# Melt dataframe so that each row is an individual grade for a student and date
attendance = attendance.melt(id_vars=['student'], var_name='date', value_name='absent')

attendance.head()

,student,date,absent
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [7]:
# Find avg of each student's attendance grade
attendance.groupby('student').absent.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: absent, dtype: float64

# 2. Coffee Levels

## A. Read the coffee_levels.csv file.

In [8]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')

coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


## B. Transform the data so that each carafe is in it's own column.

In [10]:
coffee.pivot(index='hour', columns='coffee_carafe')

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

## C. Is this the best shape for the data?

- While this data is much easier for a human to read, it is better for ML to see the data in a "tidy" form where each row is the individual target variable.

# 3. Cake Recipes

## A. Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [11]:
cake = pd.read_csv('untidy-data/cake_recipes.csv')

cake.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


## B. Tidy the data as necessary.

In [12]:
# Melt dataframe so that target variable (tastiness) is on each individual observation
cake = cake.melt(id_vars=['recipe:position'], var_name='temperature', value_name='tastiness')
    
cake.head()

,recipe:position,temperature,tastiness
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [14]:
# Split recipe and position into seperate features
cake[['recipe', 'position']]= cake['recipe:position'].str.split(':', expand = True)
    
cake.head()

,recipe:position,temperature,tastiness,recipe,position
0,a:bottom,225,61.738655,a,bottom
1,a:top,225,51.709751,a,top
2,b:bottom,225,57.095320,b,bottom
3,b:top,225,82.455004,b,top
4,c:bottom,225,96.470207,c,bottom


In [15]:
# Drop the combined column as it is no longer needed 
cake = cake.drop(columns='recipe:position')

cake.head()

,temperature,tastiness,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom


In [16]:
# Pivot data 
cake_tidy = cake.pivot(index=['recipe', 'position'], columns='temperature')

cake_tidy

tastiness                                 
temperature            225        250        275        300
recipe position                                            
a      bottom    61.738655  53.912627  74.414730  98.786784
       top       51.709751  52.009735  68.576858  50.228470
b      bottom    57.095320  61.904369  61.196980  99.248541
       top       82.455004  95.224151  98.594881  58.169349
c      bottom    96.470207  52.001358  92.893227  65.473084
       top       71.306308  82.795477  92.098049  53.960273
d      bottom    52.799753  58.670419  51.747686  56.183110
       top       96.873178  76.101363  59.571620  50.971626

In [17]:
cake_tidy.reset_index(inplace=True)


cake_tidy

recipe position  tastiness                                 
temperature                        225        250        275        300
0                a   bottom  61.738655  53.912627  74.414730  98.786784
1                a      top  51.709751  52.009735  68.576858  50.228470
2                b   bottom  57.095320  61.904369  61.196980  99.248541
3                b      top  82.455004  95.224151  98.594881  58.169349
4                c   bottom  96.470207  52.001358  92.893227  65.473084
5                c      top  71.306308  82.795477  92.098049  53.960273
6                d   bottom  52.799753  58.670419  51.747686  56.183110
7                d      top  96.873178  76.101363  59.571620  50.971626

## C. Which recipe, on average, is the best? recipe b

In [29]:
cake.groupby('recipe').tastiness.mean().sort_values().tail(1)

recipe
b    76.736074
Name: tastiness, dtype: float64

## D. Which oven temperature, on average, produces the best results? 275

In [31]:
cake.groupby('temperature').tastiness.mean().sort_values().tail(1)

temperature
275    74.886754
Name: tastiness, dtype: float64

## E. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [33]:
cake.sort_values(by = 'tastiness').tail(1)

,temperature,tastiness,recipe,position
26,300,99.248541,b,bottom
